In [2]:
pip install msedge.selenium_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.6/904.6 kB 15.0 MB/s eta 0:00:00


In [5]:
pip install zenrows

In [6]:
import csv
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions
import pandas as pd
import csv
import json
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd
import time
from zenrows import ZenRowsClient

In [7]:
q = input("Enter product name (Searching on flipkart)")
q = q.replace(" ","+")
test = []
names = []
flipPrices = []
prodNames = []
info,price = [],[]
client = ZenRowsClient("ee972034eff5595e3cbc074967dd4da3b2fdfbe1")
url = "https://flipkart.com/search?q="
file_name = q.replace(" ", "_")
# response recieved in bytes
resp = client.get(url+q)
print(resp.status_code)
# parsing response content using BeautifulSoup class, so that we can perform operations on it.
parsed_html = bs(resp.content, 'html.parser')
# data cleaning
raw_data = parsed_html.find("script", attrs={"id":"is_script"})
data = raw_data.contents[0].replace("window.__INITIAL_STATE__ = ","").replace(";","")
json_data = json.loads(data)
req_data = json_data["pageDataV4"]["page"]["data"]["10003"]

data_list = []
try:
    for i in range(1, len(req_data)):
        d = {}
        jd = req_data[i]["widget"]["data"]["products"]
        for j in range(len(jd)):
            jd2 = jd[j]["productInfo"]["value"]

            d["title"] = jd2["titles"]["title"]
            d["keySpecs"] = jd2["keySpecs"]
            d["rating"] = jd2["rating"]["average"]
            d["ratingCount"] = jd2["rating"]["count"]
            d["price"] = jd2["pricing"]["finalPrice"]["value"]
            d["url"] = jd2["smartUrl"]
        data_list.append(d)


except:
    pass
# dumping data to result.json file
#     print(list(data_list))
with open("flipkart"+'.json', 'w') as fp:
     json.dump(data_list, fp)

# Now let us write our data to csv file
data_file = open("flipkart"+'.csv', 'w')

# create the csv writer object
csv_writer = csv.writer(data_file)

# Counter variable used for writing
# headers to the CSV file
count = 0

for data in data_list:
    if count == 0:

        # Writing headers of CSV file
        header = data.keys()
        csv_writer.writerow(header)
        count += 1
    # Writing data of CSV file
    csv_writer.writerow(data.values())


with open('flipkart.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    rows = list(reader)
i,j = 0,2
while i in range(len(rows)):
    try:
        name = rows[j][0]
        names.append(name)
        i += 1
        j += 2
    except:
            break

Enter product name (Searching on flipkart)sugar
200


In [8]:

with open('flipkart.csv') as csv_file:
    reader = csv.reader(csv_file, delimiter=',')
    rows = list(reader)
i,j = 0,2
while i in range(len(rows)):
    try:
        name = rows[j][0]
        names.append(name)
        i += 1
        j += 2
    except:
            break


print("Best results",len(names))
if len(names) <= 10:
    flipkart_url = "https://www.flipkart.com/search?q=" + q
    print(flipkart_url)
    uClient = client.get(flipkart_url).content
    flipkartPage = uClient
    flipkart_html = bs(flipkartPage, "html.parser")
    bigboxes = flipkart_html.find_all("a", {"class": "s1Q9rs"})
    soup = BeautifulSoup(flipkartPage, 'html.parser')
    info = soup.select("[class~=s1Q9rs]")
    flipPrices = soup.select("[class =_30jeq3]")
    prodNames = [i.get('title') for i in info]
    names = prodNames
    df = pd.DataFrame(list(zip(prodNames, flipPrices)),
                columns =['product_name', 'Flipkart_price'])
    df.to_csv('flipkart.csv')
    print(df)
else:
    with open('flipkart.csv') as csv_file:
        reader = csv.reader(csv_file, delimiter=',')
        rows = list(reader)
    i,j = 0,2
    while i in range(len(rows)):
        try:
            price = rows[j][4]
            flipPrices.append(price)
            i += 1
            j += 2
        except:
            break
    df = pd.DataFrame(list(zip(names, flipPrices)),
            columns =['Product_name', 'Flipkart_price'])

    print(df)
    df.to_csv('flipkart.csv')

data_file.close()

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

flipkart=''
#ebay=''
amazon=''
#olx=''

amazonlist = []
amazonName = []
i = 0
while i in range(len(names)):
    print(names[i])
    def amazon(name):
            try:
                global amazon
                name = " ".join(name.split(' ')[0:2])
                name1 = name.replace(" ","-")
                name2 = name.replace(" ","+")
                amazon=f'https://www.amazon.in/{name1}/s?k={name2}'
                res = client.get(f'https://www.amazon.in/{name1}/s?k={name2}',headers=headers)
                print("\nSearching in amazon:")
                soup = BeautifulSoup(res.text,'html.parser')
                amazon_page = soup.select('.a-color-base.a-text-normal')
                amazon_page_length = int(len(amazon_page))
                for i in range(0,amazon_page_length):
                    name = name.upper()
                    amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                    if name in amazon_name[0:20]:
                        amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                        amazon_price = soup.select('.a-price-whole')[i].getText().strip().upper()
                        amazonlist.append(amazon_price)
                        print("Amazon:")
                        print(amazon_name)
                        amazonName.append(amazon_name)
                        print("₹"+amazon_price)
                        print("-----------------------")
                        break
                    else:
                        i+=1
                        i=int(i)
                        if i==amazon_page_length:
                            print("amazon : No product found!")
                            print("-----------------------")
                            amazon_price = '0'
                            amazonlist.append(amazon_price)
                            amazonName.append("No similar product")
                            break

                return amazon_price
            except:
                print("amazon: No product found!")
                print("-----------------------")
                amazon_price = '0'
                amazonlist.append(amazon_price)
                amazonName.append("No similar product")
            return amazon_price
    amazon_price = amazon(names[i])
    flipkart=''
    ebay=''
    croma=''
    amazon=''
    olx=''
    i += 1

flip = flipPrices
idk = []
for i in range(len(flip)):
#     x =
    try:
        x = flip[i].text.replace('₹','')
        print(x)
        idk.append(x)
    except:
        idk = test
df = pd.DataFrame(list(zip(names,idk,amazonName, amazonlist)),
            columns =["Product_name_Flipkart","Flipkart_price",'Product_name_Amazon', 'Amazon_price'])
print(df)
df.to_csv('flipkartandamazon.csv')

Best results 0
https://www.flipkart.com/search?q=sugar
                                         product_name Flipkart_price
0                    I'm lite 50% Less Calories Sugar         [₹252]
1   DiaBliss Diabetic Friendly 1 Kg Reusable Jar S...         [₹544]
2   organic forest Country Side Brown Sugar , Mine...         [₹226]
3                  Sri Sri Tattva Organic Brown Sugar          [₹82]
4   organic forest Dhaga Mishri, 900g (Pure Thread...         [₹212]
5             PRO ORGANIC LIFE Dhaga Mishri 1kg Sugar         [₹242]
6                         DiaBliss Reusable Jar Sugar         [₹294]
7   Sri Sri Tattva Brown Sugar, 500g each ( Pack o...         [₹202]
8   Shudh Online Tal Mishri/Palm Candy/Palm Sugar/...         [₹110]
9   organic forest Country Side Brown Sugar 2Kgs ,...         [₹342]
10                   DiaBliss 500g Standy Pouch Sugar         [₹160]
11  DiaBliss Diabetic Friendly Herbal Cane Sugar F...         [₹299]
12  Sri Sri Tattva Brown Sugar, 1kg each ( Pack 

In [9]:
df = pd.DataFrame(list(zip(amazonName, amazonlist)),
               columns =['Product_name', 'Amazon_price'])

print(df)

                                         Product_name Amazon_price
0               I'M LITE 50% LESS CALORIES SUGAR 1 KG          245
1   DIABLISS DIABETIC FRIENDLY HERBAL CANE SUGAR -...          274
2   ORGANIC FOREST 100% VEGAN AMLA, BHRINGRAJ & SH...          445
3   SRI SRI TATTVA SHUDDHTA KA NAAM SUDANTA AYURVE...          294
4   ORGANIC FOREST 100% VEGAN AMLA, BHRINGRAJ & SH...          445
5   J.K PRO ORGANIC FERTILIZER PLANT GROWTH PROMOT...          238
6                                  No similar product            0
7   SRI SRI TATTVA COW GHEE - 1 LITRE (PACK OF 1) ...          294
8   SHUDH ONLINE BLACK KAUNCH BEEJ POWDER, KONCH S...          239
9   ORGANIC FOREST 100% VEGAN AMLA, BHRINGRAJ & SH...          445
10                                 No similar product            0
11  DIABLISS DIABETIC FRIENDLY HERBAL CANE SUGAR -...          274
12  SRI SRI TATTVA COW GHEE - 1 LITRE (PACK OF 1) ...          294
13                                 No similar product         

In [10]:
test = flipPrices

In [11]:
flip = flipPrices

In [12]:
idk = []

In [13]:
flip = flipPrices
idk = []
for i in range(len(flip)):
#     x =
    try:
        x = flip[i].text.replace('₹','')
        print(x)
        idk.append(x)
    except:
        idk = test

252
544
226
82
212
242
294
202
110
342
160
299
401
325
219
400
449
1,002
188
419
110
570
665
299
300
143
246
161
321
229
490
213
199
360
99
399
749
189
219
635


In [14]:
import csv
# opening the csv file in 'w+' mode
file = open('flipkartandamazon.csv', 'w+', newline ='')
# writing the data into the file
with file:
    write = csv.writer(file)
    write.writerows(names)
    write.writerows(idk)
    write.writerows(amazonName)
    write.writerows(amazonlist)

In [15]:
df = pd.DataFrame(list(zip(names,idk,amazonName, amazonlist)),
               columns =["Product_name_Flipkart","Flipkart_price",'Product_name_Amazon', 'Amazon_price'])

df

,Product_name_Flipkart,Flipkart_price,Product_name_Amazon,Amazon_price
0,I'm lite 50% Less Calories Sugar,252,I'M LITE 50% LESS CALORIES SUGAR 1 KG,245
1,DiaBliss Diabetic Friendly 1 Kg Reusable Jar S...,544,DIABLISS DIABETIC FRIENDLY HERBAL CANE SUGAR -...,274
2,"organic forest Country Side Brown Sugar , Mine...",226,"ORGANIC FOREST 100% VEGAN AMLA, BHRINGRAJ & SH...",445
3,Sri Sri Tattva Organic Brown Sugar,82,SRI SRI TATTVA SHUDDHTA KA NAAM SUDANTA AYURVE...,294
4,"organic forest Dhaga Mishri, 900g (Pure Thread...",212,"ORGANIC FOREST 100% VEGAN AMLA, BHRINGRAJ & SH...",445
5,PRO ORGANIC LIFE Dhaga Mishri 1kg Sugar,242,J.K PRO ORGANIC FERTILIZER PLANT GROWTH PROMOT...,238
6,DiaBliss Reusable Jar Sugar,294,No similar product,0
7,"Sri Sri Tattva Brown Sugar, 500g each ( Pack o...",202,SRI SRI TATTVA COW GHEE - 1 LITRE (PACK OF 1) ...,294
8,Shudh Online Tal Mishri/Palm Candy/Palm Sugar/...,110,"SHUDH ONLINE BLACK KAUNCH BEEJ POWDER, KONCH S...",239
9,"organic forest Country Side Brown Sugar 2Kgs ,...",342,"ORGANIC FOREST 100% VEGAN AMLA, BHRINGRAJ & SH...",445


In [16]:
df.to_csv('flipkartandamazon.csv')

In [17]:
from flask import Flask, redirect, url_for, render_template, request, flash
from flask import request
import csv
from bs4 import BeautifulSoup
from msedge.selenium_tools import Edge, EdgeOptions
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import time
app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/login", methods=["POST", "GET"])
def login():
    if request.method == "POST":
        test = []
        names = []
        flipPrices = []
        prodNames = []
        info,price = [],[]
        url = "https://flipkart.com/search?q="
        q = request.form["nm"]
        q = q.replace(" ","+")
        # response recieved in bytes
        resp = requests.get(url+q)
        # parsing response content using BeautifulSoup class, so that we can perform operations on it.
        parsed_html = bs(resp.content, 'html.parser')
        # data cleaning
        raw_data = parsed_html.find("script", attrs={"id":"is_script"})
        data = raw_data.contents[0].replace("window.__INITIAL_STATE__ = ","").replace(";","")
        json_data = json.loads(data)
        req_data = json_data["pageDataV4"]["page"]["data"]["10003"]   #[10]["widget"]["data"]["products"][3]["productInfo"]
        #req_json_data = json_data["seoMeta"]["answerBox"]["data"]["renderableComponents"][0]["value"]["data"]

        data_list = []
        try:
            for i in range(1, len(req_data)):
                d = {}
                jd = req_data[i]["widget"]["data"]["products"]
                # print(len(jd))
                # print("i: ", i, end="\n")
                for j in range(len(jd)):
                    jd2 = jd[j]["productInfo"]["value"]

                    d["title"] = jd2["titles"]["title"]
                    d["keySpecs"] = jd2["keySpecs"]
                    d["rating"] = jd2["rating"]["average"]
                    d["ratingCount"] = jd2["rating"]["count"]
                    d["price"] = jd2["pricing"]["finalPrice"]["value"]
        #                 d["warranty"] = jd2["warrantySummary"]
                    d["url"] = jd2["smartUrl"]
                data_list.append(d)

        except:
            pass
        # dumping data to result.json file
        #     print(list(data_list))
        with open("flipkart"+'.json', 'w') as fp:
            json.dump(data_list, fp)

        # Now let us write our data to csv file
        data_file = open("flipkart"+'.csv', 'w')

        # create the csv writer object
        csv_writer = csv.writer(data_file)

        # Counter variable used for writing
        # headers to the CSV file
        count = 0

        for data in data_list:
            if count == 0:

                # Writing headers of CSV file
                header = data.keys()
                csv_writer.writerow(header)
                count += 1
            # Writing data of CSV file

            csv_writer.writerow(data.values())


        with open('flipkart.csv') as csv_file:
            reader = csv.reader(csv_file, delimiter=',')
            rows = list(reader)
        i,j = 0,2
        while i in range(len(rows)):
            try:
                name = rows[j][0]
                #             name = " ".join(name.split(' ')[0:2])
        #         print(name)
        #         print("name = ",name)
                names.append(name)
                i += 1
                j += 2
            except:
                    break


        print("Best results",len(names))
        #     print(names,len(names))
        if len(names) <= 10:
            flipkart_url = "https://www.flipkart.com/search?q=" + q
            print(flipkart_url)
            uClient = uReq(flipkart_url)
            flipkartPage = uClient.read()
            uClient.close()
            flipkart_html = bs(flipkartPage, "html.parser")
            soup = BeautifulSoup(flipkartPage, 'html.parser')
            info = soup.select("[class~=s1Q9rs]")
            flipPrices = soup.select("[class =_30jeq3]")
            prodNames = [i.get('title') for i in info]
            names = prodNames
            df = pd.DataFrame(list(zip(prodNames, flipPrices)),
                        columns =['product_name', 'Flipkart_price'])
            df.to_csv('flipkart.csv')
            print(df)

        else:
            with open('flipkart.csv') as csv_file:
                reader = csv.reader(csv_file, delimiter=',')
                rows = list(reader)
            i,j = 0,2
            while i in range(len(rows)):
                try:
                    price = rows[j][4]
        #             price = price[i].text
        #             print("price = ",price)
                    flipPrices.append(price)
                    i += 1
                    j += 2
                except:
                    break
            df = pd.DataFrame(list(zip(names, flipPrices)),
                    columns =['Product_name', 'Flipkart_price'])

            print(df)
            df.to_csv('flipkart.csv')

        data_file.close()

        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'}

        amazon=''
        amazonlist = []
        amazonName = []
        i = 0
        while i in range(len(names)):
            print(names[i])
            def amazon(name):
                    try:
                        global amazon
                        name = " ".join(name.split(' ')[0:2])
                        name1 = name.replace(" ","-")
                        name2 = name.replace(" ","+")
                        amazon=f'https://www.amazon.in/{name1}/s?k={name2}'
                        res = requests.get(f'https://www.amazon.in/{name1}/s?k={name2}',headers=headers)
                        print("\nSearching in amazon:")
                        soup = BeautifulSoup(res.text,'html.parser')
                        amazon_page = soup.select('.a-color-base.a-text-normal')
                        amazon_page_length = int(len(amazon_page))
                        for i in range(0,amazon_page_length):
                            name = name.upper()
                            amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                            if name in amazon_name[0:20]:
                                amazon_name = soup.select('.a-color-base.a-text-normal')[i].getText().strip().upper()
                                amazon_price = soup.select('.a-price-whole')[i].getText().strip().upper()
                                amazonlist.append(amazon_price)
                                print("Amazon:")
                                print(amazon_name)
                                amazonName.append(amazon_name)
                                print("₹"+amazon_price)
                                print("-----------------------")
                                break
                            else:
                                i+=1
                                i=int(i)
                                if i==amazon_page_length:
                                    print("amazon : No product found!")
                                    print("-----------------------")
                                    amazon_price = '0'
                                    amazonlist.append(amazon_price)
                                    amazonName.append("No similar product")
                                    break

                        return amazon_price
                    except:
                        print("amazon: No product found!")
                        print("-----------------------")
                        amazon_price = '0'
                        amazonlist.append(amazon_price)
                        amazonName.append("No similar product")
                    return amazon_price
            amazon_price = amazon(names[i])
            amazon=''
            olx=''
            i += 1
        test = flipPrices
        flip = flipPrices
        idk = []
        for i in range(len(flip)):
        #     x =
            try:
                x = flip[i].text.replace('₹','')
                print(x)
                idk.append(x)
            except:
                idk = test
        df = pd.DataFrame(list(zip(names,idk,amazonName, amazonlist)),
                    columns =["Product_name_Flipkart","Flipkart_price",'Product_name_Amazon', 'Amazon_price'])
        print(df)
        df.to_csv('flipkartandamazon.csv')
        filename = 'flipkartandamazon.csv'
        data = pd.read_csv(filename, header=0)
        stocklist = list(data.values)
        return render_template('login.html', stocklist=stocklist)
    else:
        return render_template("login.html")


@app.route('/stocks')
def Stocks():
    key = "YO"
    filename = 'flipkartandamazon.csv'
    data = pd.read_csv(filename, header=0)
    stocklist = list(data.values)
    return render_template('stocks.html', stocklist=stocklist, key = key)


@app.route("/<usr>")
def user(usr):

    return f"<h1>{usr}</h1>"

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
